## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2141 entries, 0 to 2140
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Seas    2141 non-null   int64  
 1   Player  2141 non-null   object 
 2   Pos     2141 non-null   object 
 3   Age     2141 non-null   int64  
 4   Tm      2141 non-null   object 
 5   G       2141 non-null   int64  
 6   GS      2141 non-null   int64  
 7   MP      2141 non-null   float64
 8   FG      2141 non-null   float64
 9   FGA     2141 non-null   float64
 10  FG%     2131 non-null   float64
 11  3P      2141 non-null   float64
 12  3PA     2141 non-null   float64
 13  3P%     1967 non-null   float64
 14  2P      2141 non-null   float64
 15  2PA     2141 non-null   float64
 16  2P%     2110 non-null   float64
 17  eFG%    2131 non-null   float64
 18  FT      2141 non-null   float64
 19  FTA     2141 non-null   float64
 20  FT%     2037 non-null   float64
 21  ORB     2141 non-null   float64
 22  

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [10]:
# answer goes here
nba = df.loc[(df['GS'] >= 20) & (df['MP'] >= 10)]

nba.info()




<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 3 to 2139
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Seas    804 non-null    int64  
 1   Player  804 non-null    object 
 2   Pos     804 non-null    object 
 3   Age     804 non-null    int64  
 4   Tm      804 non-null    object 
 5   G       804 non-null    int64  
 6   GS      804 non-null    int64  
 7   MP      804 non-null    float64
 8   FG      804 non-null    float64
 9   FGA     804 non-null    float64
 10  FG%     804 non-null    float64
 11  3P      804 non-null    float64
 12  3PA     804 non-null    float64
 13  3P%     771 non-null    float64
 14  2P      804 non-null    float64
 15  2PA     804 non-null    float64
 16  2P%     804 non-null    float64
 17  eFG%    804 non-null    float64
 18  FT      804 non-null    float64
 19  FTA     804 non-null    float64
 20  FT%     804 non-null    float64
 21  ORB     804 non-null    float64
 22  D

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [16]:
# answer goes here
X = nba.loc[:,['PTS','TRB','AST','STL','BLK','TOV']]

X.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 3 to 2139
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PTS     804 non-null    float64
 1   TRB     804 non-null    float64
 2   AST     804 non-null    float64
 3   STL     804 non-null    float64
 4   BLK     804 non-null    float64
 5   TOV     804 non-null    float64
dtypes: float64(6)
memory usage: 44.0 KB


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [24]:
# answer goes here
scale = StandardScaler()
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)

Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [26]:
# answer goes here
kmeans = KMeans(n_clusters=3)

kmeans.fit_predict(X_scale)

centers = pd.DataFrame(kmeans.cluster_centers_,columns=X.columns)

centers_inverse = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
centers_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,10.505405,3.949775,2.063739,0.804730,0.374324,1.235135
1,13.735359,8.605525,1.914917,0.777348,1.230387,1.668508
2,19.491620,5.293855,5.934078,1.354190,0.486034,2.797765


One group (probably more guards) seems to contain the higher scoring players who also have more assists, steals, and turnovers. Another group (probably more centers and forwards) has players with many more rebounds and blocks. The final group doesn't stand out in any category. This is probably your less skillied role players. 

Experiment with different values of K. Do any further interesting archetypes come out?

In [32]:
# answer goes here
kmeans = KMeans(n_clusters=6)

kmeans.fit_predict(X_scale)

centers = pd.DataFrame(kmeans.cluster_centers_,columns=X.columns)

centers_inverse = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
centers_inverse.style.background_gradient()



,PTS,TRB,AST,STL,BLK,TOV
0,17.755000,10.473333,2.343333,0.921667,1.743333,2.140000
1,7.707853,3.487958,1.405236,0.602094,0.354974,0.905759
2,13.139732,4.013839,2.829911,1.032589,0.333036,1.562054
3,18.606923,4.612308,5.580769,1.273077,0.386154,2.571538
4,23.452632,7.907895,7.589474,1.660526,0.805263,3.760526
5,11.481988,7.236646,1.666460,0.684472,0.919876,1.383230


At higher Ks (4-7) you start to see sub sets of similiar groups. For example at a K of 6 you have a group (5) of players that scores more per game in 4 of 6 categories than any other gourp. You also see a 2nd group (3) that is similiar, but scores less than the 1st group. However, this group is 2nd in scoring in the same 4 of 6 categories as the first. Just like with a K of 3 we discover an archetype of higher scoring, assists, steals, and turnover, but a higher and lower scoring version of that archetype emerges.

You still see a group (0) that clusters around the best rebounds and blocks per game numbers. But you also a see a worse ppg group (5) that has similiar charateristics. There seems to be a higher and lower scoring version of the block/rebound archetype. You also see a group (1) that has really low ppg averages in all categories, just like with a K of 3. 

Overall, you see sub clusters of similiar groups from what we saw with a K of 3.